In [1]:
from qsharp.chemistry import load_broombridge, load_fermion_hamiltonian, load_input_state, encode

Preparing Q# environment...
.Adding package microsoft.quantum.chemistry.jupyter.

In [2]:
filename = 'data/hydrogen_0.2.yaml'
broombridge_data =  load_broombridge(filename)

In [3]:
# Retrieve basis set and geometry used to generate the input data
basis_set = broombridge_data.problem_description[0].basis_set
geometry = broombridge_data.problem_description[0].geometry

# Retrieve the nuclear repulsion and the one-electron integrals (Mulliken convention)
nuclear_repulsion = broombridge_data.problem_description[0].coulomb_repulsion['Value']
one_electron_integrals =  broombridge_data.problem_description[0].hamiltonian['OneElectronIntegrals']['Values']

print("nuclear_repulsion = ", nuclear_repulsion)
print("one_electron_integrals = ", one_electron_integrals)

nuclear_repulsion =  0.713776188
one_electron_integrals =  [([1, 1], -1.252477495), ([2, 2], -0.475934275)]


In [4]:
ferm_hamiltonian = broombridge_data.problem_description[0].load_fermion_hamiltonian()
print("ferm_hamiltonian ::", ferm_hamiltonian)

ferm_hamiltonian :: <qsharp.chemistry.FermionHamiltonian object at 0x7f5590d52590>


In [5]:
input_state = load_input_state(filename, "UCCSD |G>")
print("input_state ::", input_state)

input_state :: <qsharp.chemistry.InputState object at 0x7f5590d56750>


In [6]:
jw_hamiltonian = encode(ferm_hamiltonian, input_state)
print("jw_hamiltonian :: \n", jw_hamiltonian)

jw_hamiltonian :: 
 (4, ([([0], [0.17120128499999998]), ([1], [0.17120128499999998]), ([2], [-0.222796536]), ([3], [-0.222796536])], [([0, 1], [0.1686232915]), ([0, 2], [0.12054614575]), ([0, 3], [0.16586802525]), ([1, 2], [0.16586802525]), ([1, 3], [0.12054614575]), ([2, 3], [0.1743495025])], [], [([0, 1, 2, 3], [0.0, -0.0453218795, 0.0, 0.0453218795])]), (3, [((0.001, 0.0), [2, 0]), ((-0.001, 0.0), [3, 1]), ((0.001, 0.0), [2, 3, 1, 0]), ((1.0, 0.0), [0, 1])]), -0.09883444600000002)


In [7]:
def get_var_params(jw_hamiltonian):
    """ Retrieve the values of variational parameters from the jw_hamiltonian object """
    _, _, input_state, _ = jw_hamiltonian
    _, var_params = input_state
    params = [param for ((param, _), _) in var_params]
    return params[:-1]

var_params = get_var_params(jw_hamiltonian)
print(var_params)

[0.001, -0.001, 0.001]


In [25]:
import qsharp

# It is possible to create a Python object to represent a
# Q# callable from the chemistry library
estimate_energy = qsharp.QSharpCallable("Microsoft.Quantum.Chemistry.JordanWigner.VQE.EstimateEnergy", "")

# The Q# operation can then be called through the simulate method
# A large number of samples is selected for high accuracy
energy = estimate_energy.simulate(jwHamiltonian=jw_hamiltonian, nSamples=1e18)

print("Energy evaluated at {0} : {1} \n".format(var_params, energy))

Energy evaluated at [0.001, -0.001, 0.001] : -1.116320683389766 



In [9]:
def set_var_params(var_params, jw_hamiltonian):
    """ Set variational parameters stored in the JW data-structure to the desired values"""
    # Unpack data structure
    a1, a2, input_state, a3 = jw_hamiltonian
    b1, amps = input_state
    # Unpack and overwrite variational parameters
    new_amps = [((var_params[i], 0.0), amps[i][1]) for i in range(len(var_params))]
    new_amps.append(amps[-1])
    # Re-pack the data structure
    input_state = (b1, new_amps)
    jw_hamiltonian = (a1, a2, input_state, a3)
    return jw_hamiltonian

In [10]:
def energy_eval_wrapper(var_params, jw_hamiltonian, n_samples):
    """
        A wrapper whose signature is compatible with the use of scipy optimizers,
        calling the Q# energy_evalaution from the Microsoft Chemistry library
    """

    # NumPy arrays are currently not supported by the Python interops
    # This ensures that neither the user nor SciPy call the energy evaluation function with a NumPy array
    var_params = list(var_params)

    # Set the varational parameters to the right values in the jw_hamiltonian object
    jw_hamiltonian = set_var_params(var_params, jw_hamiltonian)

    # Estimate the energy
    energy = estimate_energy.simulate(jwHamiltonian=jw_hamiltonian, nSamples=1e18)

    print("Energy evaluated at {0} : {1} \n".format(var_params, energy))
    return energy

In [11]:
from scipy.optimize import minimize

def VQE(initial_var_params, jw_hamiltonian, n_samples):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """

    opt_result = minimize(energy_eval_wrapper,
                          initial_var_params,
                          args=(jw_hamiltonian, n_samples),
                          method="COBYLA",
                          tol=0.000001,
                          options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05})

    return opt_result

In [13]:
# Run VQE and print the results of the optimization process
# A large number of samples is selected for higher accuracy
opt_result = VQE(var_params, jw_hamiltonian, n_samples=1e18)
print(opt_result)

Energy evaluated at [0.001, -0.001, 0.001] : -1.1163206833694206 

Energy evaluated at [0.051000000000000004, -0.001, 0.001] : -1.1143693476093461 

Energy evaluated at [0.001, 0.049, 0.001] : -1.1144494157068259 

Energy evaluated at [0.001, -0.001, 0.051000000000000004] : -1.0941301536809216 

Energy evaluated at [-0.0033645025138452564, -0.00518541625925021, -0.04863298710075229] : -1.1305266764099717 

Energy evaluated at [-0.01021579424489009, -0.011755582668140094, -0.09772364522402063] : -1.1366498444335464 

Energy evaluated at [-0.04496727191940477, -0.046255806787029896, -0.1078265821823552] : -1.1331696225130465 

Energy evaluated at [0.007410274691206744, -0.029484488285011885, -0.0978108202664006] : -1.1362581506365759 

Energy evaluated at [0.0067936479471139375, 0.011608616182394326, -0.13852552126409456] : -1.1360580838371521 

Energy evaluated at [-0.001711073148888076, -7.348324287288419e-05, -0.11812458324405761] : -1.1372266410892609 

Energy evaluated at [0.0110109

In [14]:
# Print difference with exact FCI value known for this bond length
fci_value = -1.1372704220924401
print("Difference with exact FCI value :: ", abs(opt_result.fun - fci_value))

Difference with exact FCI value ::  8.032904119659179e-09


In [17]:
num_qubits, hamiltonian_term_list, input_state, energy_offset = jw_hamiltonian

In [19]:
var_params = get_var_params(jw_hamiltonian)

In [29]:
qsharp.reload()

In [32]:
num_qubits

4

In [38]:
import qsharp
qsharp.reload()
from Microsoft.Quantum.Samples.Chemistry.VariationalQuantumEigensolver import EstimateEnergy1

# It is possible to create a Python object to represent a
# Q# callable from the chemistry library
# estimate_energy2 = qsharp.QSharpCallable("Microsoft.Quantum.Samples.Chemistry.VariationalQuantumEigensolver.EstimateEnergy", "")

# The Q# operation can then be called through the simulate method
# A large number of samples is selected for high accuracy
energy = EstimateEnergy1.simulate(
    nQubits=num_qubits,
    hamiltonianTermList=hamiltonian_term_list, 
    inputState=input_state,
    energyOffset=energy_offset,
    nSamples=1e18)

print("Energy evaluated at {0} : {1} \n".format(var_params, energy))

Unhandled exception. System.NotImplementedException: The method or operation is not implemented.
 ---> Microsoft.Quantum.Samples.Chemistry.VQE.StatePreparation.PrepareSingleConfigurationalStateSingleSiteOccupation on /workspaces/Quantum/samples/chemistry/VariationalQuantumEigensolver/StatePreparation.qs:line 0
   at Microsoft.Quantum.Canon.ApplyControlledOnBitString on /workspaces/Quantum/samples/chemistry/VariationalQuantumEigensolver/D:\a\1\s\submodules\QuantumLibraries\Standard\src\Canon\Utils\ControlledOnBitString.qs:line 0
   at Microsoft.Quantum.Canon.ApplyControlledOnInt on /workspaces/Quantum/samples/chemistry/VariationalQuantumEigensolver/D:\a\1\s\submodules\QuantumLibraries\Standard\src\Canon\Utils\ControlledOnBitString.qs:line 0
   at Microsoft.Quantum.Canon.MultiplexOperationsBruteForceFromGenerator on /workspaces/Quantum/samples/chemistry/VariationalQuantumEigensolver/D:\a\1\s\submodules\QuantumLibraries\Standard\src\Canon\Multiplexer.qs:line 0
   at Microsoft.Quantum.Samp